# Instructions on Use
Run all cells sequntially.

-Yang

In [61]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
pwd = os.getcwd()
print(pwd)

/Users/edwardyao/Documents/gatech_cse6242/group_project


### Notes on Data
Only data files used are "predictions_on_2011.csv" and "predictions_on_2021.csv". You may need to edit the path to load the file

In [64]:
# import predictions and original data to get city names and neighborhoods

preds_2011 = pd.read_csv('predictions_on_2011.csv')
preds_2021 = pd.read_csv('predictions_on_2021.csv')

og_data = pd.read_csv(pwd+'/data/normalized_percents.csv')
og_data.columns = og_data.columns.str.lower()
#og_data = og_data[og_data['year'] != 2001]

og2011 = og_data[og_data['year'] == 2011].set_index("msoa code")
og2021 = og_data[og_data['year'] == 2021].set_index("msoa code")

In [80]:
# subset predictions to only gentrified 

gentrified_2021 = preds_2011[preds_2011['sd_change'] >= 1.0]
gentrified_2031 = preds_2021[preds_2021['sd_change'] >= 1.0]

num_gent_2021 = len(gentrified_2021)
num_gent_2031 = len(gentrified_2031)

In [129]:
def top_var_analysis(df, year):
    
    df = df.set_index("msoa code")
    
    pattern = "var[123]"
    comp = re.compile(pattern)
    var_cols = [c for c in df.columns.tolist() if comp.match(c)]
    
    
    var_diff = {} # dict with var 1-3 has top 3
    #var_stats = {} # mean, sd, median of shapley values
    top_perc = {} # keys are var 1-3; vals are df of perc of each top 3
    
    if year == 2011:
        for vc in var_cols:
            
            var_diff[vc] = {}
            
            sub = pd.DataFrame(df.loc[:, vc]) # this is now var1 or 2 or 3
            # get the top 3 by count - change to % of all gentrified
            top3 = pd.DataFrame(sub.value_counts()).iloc[:3] / num_gent_2021
            top_perc[vc] = top3
            # get % cols
            for t in top3.index: # for each var
                t = t[0]
                tmp2011 = og2011.loc[:,t] # all gentrified, but only #1 col
                tmp2021 = og2021.loc[:,t] 
                ph = sub[sub[vc] == t] # subset to select msoa by merging
                
                ph = ph.merge(tmp2011, how='inner', on='msoa code').merge(tmp2021, how='inner', on='msoa code',
                                                                         suffixes=['_2011', '_2021'])
                ph['diff'] = ph[t+'_2021'] - ph[t+'_2011'] # diff between years
                mean_diff = np.mean(ph['diff'])
                sd_diff = np.std(ph['diff'])
                median_diff = np.median(ph['diff'])
                
                var_diff[vc][t] = {'mean': round(mean_diff, 3), 'sd': round(sd_diff, 3), 
                                   'median': round(median_diff, 3)}
        print('2021 predictions...')
        for k,v in top_perc.items(): # printing the percent dfs
            print(f'For {k} top 3 features:')
            print(f'{v}')
        
        
        for k,v in var_diff.items():
            print(f'For {k}') # k is var1-3; v is dict
            for ki, vi in v.items(): # ki is feature name; vi is dict
                  print(f'{ki}\n: {vi}')

    else:
        for vc in var_cols:
            
            top_perc[vc] = {}
            
            sub = pd.DataFrame(df.loc[:, vc]) # this is now var1 or 2 or 3
            # get the top 3 by count - change to % of all gentrified
            top3 = pd.DataFrame(sub.value_counts()).iloc[:3] / num_gent_2031
            top_perc[vc] = top3
            
            for t in top3.index: # for each var
                t = t[0]
                ph = sub[sub[vc] == t] # subset to select msoa by merging
                
                
        print('2031 predictions...')
        for k,v in top_perc.items():
            print(f'{k} top 3 features:')
            print(f'{v}')


In [130]:
top_var_analysis(gentrified_2021, year=2011)

2021 predictions...
For var1 top 3 features:
                                                       count
var1                                                        
ethnic group: black, black british, black welsh...  0.547667
single family household: lone parent family: al...  0.235294
ownership: owned: owns outright                     0.046653
For var2 top 3 features:
                                    count
var2                                     
passport: africa                 0.330629
ownership: owned: owns outright  0.146045
industry: c: manufacturing       0.095335
For var3 top 3 features:
                                                  count
var3                                                   
ethnic group: mixed or multiple ethnic groups  0.219067
industry: c: manufacturing                     0.176471
passport: africa                               0.133874
For var1
ethnic group: black, black british, black welsh, caribbean or african
: {'mean': -0.011, 'sd': 0.032,

In [100]:
top_var_analysis(gentrified_2031, year=2021)

2031 predictions...
var1 top 3 features:
                                                       count
var1                                                        
ethnic group: black, black british, black welsh...  0.685775
passport: africa                                    0.225053
ownership: owned: owns outright                     0.044586
var2 top 3 features:
                                                       count
var2                                                        
passport: africa                                    0.626327
ownership: owned: owns outright                     0.110403
single family household: lone parent family: al...  0.097665
var3 top 3 features:
                                                       count
var3                                                        
single family household: lone parent family: al...  0.312102
ethnic group: mixed or multiple ethnic groups       0.299363
passport: africa                                    0.127389


In [116]:
# top 3 cities with gentrification 2021
cities_2011 = og2011[['city name']]
city_change_2021 = gentrified_2021.set_index('msoa code')[['sd_change']].merge(cities_2011, how='inner', on='msoa code')

(pd.DataFrame(city_change_2021.value_counts('city name')).iloc[:3, 0] / num_gent_2021)*100

city name
London        77.079108
Manchester     5.273834
Bristol        5.273834
Name: count, dtype: float64

In [117]:
# top 3 cities with gentrification 2031
cities_2021 = og2021[['city name']]
city_change_2031 = gentrified_2031.set_index('msoa code')[['sd_change']].merge(cities_2021, how='inner', on='msoa code')

(pd.DataFrame(city_change_2031.value_counts('city name')).iloc[:3, 0] / num_gent_2031)*100

city name
London        79.617834
Manchester     4.670913
Birmingham     3.397028
Name: count, dtype: float64